In [1]:
from ortools.linear_solver import pywraplp
import random

In [2]:
schedule = pywraplp.Solver.CreateSolver('SCIP')
P,D,T,S,C = 3,5,8,5,10

x = {}

for p in range(P):
    x[p] = {}
    for d in range(D):
        x[p][d] = {}
        for t in range(T):
            x[p][d][t] = {}
            for s in range(S):
                x[p][d][t][s] = schedule.BoolVar(f'X[{p}[{d}][{t}][{s}]')
                
def X(p,d,t,s):
    return x[p][d][t][s]

collaborators = {}
availability = 75
AvailabilityList = [True] * availability + [False] * (100-availability)

L = [] #liders
H = [] #horas de projeto

def randomCollab(C,h):
    collaborators.clear()
    for p in range(P):
        H.append(random.randint(1,h))
        L.append(random.randint(0,C-1))
        for c in range(C):
            for d in range(D):
                for t in range(T):
                    collaborators[p,c,d,t] = random.choice(AvailabilityList)

randomCollab(C,5)

A restrição:

1. Para cada dia $(D)\,$, tempo $(T)\,$ e sala $(S)\,$ existe apenas um projeto $(P)\,$.

$$\forall_d \cdot \forall_t \cdot \forall_s \cdot \quad \sum_{p<P} \space x_{p,d,t,s} \leq 1$$

In [3]:
for d in range(D):
    for t in range(T):
        for s in range(S):
            schedule.Add(sum([X(p,d,t,s) for p in range(P)]) <= 1)

Para qualquer projeto, dia e tempo, só poderá ser numa sala.

$$\forall_p \cdot \forall_d \cdot \forall_t \cdot \quad \sum_{s<S} \space x_{p,d,t,s} \leq 1$$


In [4]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            schedule.Add(sum([X(p,d,t,s) for s in range(S)]) <= 1)

A seguinte restrição vem da existencia de um $Lider$ por projeto que é obrigado a estar presente em todas as reuniões. 

$$
\forall p \cdot \forall d \cdot \forall t \cdot \forall s \cdot \space x_{p,d,t,s} \leq collaborators_{p,L_p,d,t}
$$

In [5]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            for s in range(S):
                schedule.Add(X(p,d,t,s) <= collaborators[p,L[p],d,t])

Sendo que cada projeto realiza um dado número de reuniões semanais, temos então a restrição de que para qualquer projeto, o número total de reuniões de cada projeto será igual a $H_p$(um array que terá essa informação).

$$\forall_p \cdot \sum_{d<D}  \sum_{t<T} \sum_{s<S} \space x_{p,d,t,s} == H_p$$


In [6]:
for p in range(P):
    schedule.Add(sum([X(p,d,t,s) for d in range(D) for t in range(T) for s in range(S)]) == H[p])

Colaboradores podem ou não estar presentes na reunião, no entanto a reunião só se realiza num mínimo "quorum" sendo este 50% dos colaboradores totais.

$$
\forall p \cdot \forall d \cdot \forall t \cdot \forall s \cdot x_{p,d,t,s} \leq (\dfrac {\sum_{c} collaborators_{p,c,d,t}}{C})
$$

In [7]:
for p in range(P):
    for d in range(D):
        for t in range(T):
            for s in range(S):
                schedule.Add(X(p,d,t,s) <= (sum([collaborators[p,c,d,t] for c in range(C)])/C))

Para maximizar o número de reuniões efetivamente realizadas, maximiza-se a soma do numero de reuniões semanais portanto $H$.

In [8]:
#schedule.Maximize(sum(H))

Para minimizar o número médio de reuniões por participante, minimiza-se ?

In [9]:
#schedule.Minimize()

In [10]:
status = schedule.Solve()

if (status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE):
    for p in range(P):
        print(F'#Lista de reuniões do projeto {p}:')
        for d in range(D):
            for t in range(T):
                for s in range(S):
                    if x[p][d][t][s].solution_value() != 0:
                        print(F'Dia: {d}, Horas: {t}, Sala: {s}')